# Kili Tutorial: Importing predictions
In this tutorial, we will show how to import predictions (pre-annotations) into Kili in order to help annotators and accelerate the whole annotation process. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way, but also to dive into the actual process of iteratively developing real applications in Kili.

Additionally:

For an overview of Kili, visit kili-technology.com You can also check out the Kili documentation https://kili-technology.github.io/kili-docs. Our goal is to export labels that can predict whether an image contains a Porsche or a Tesla.

The tutorial is divided into four parts:

1. Understanding the different types of labels
2. Understanding the data model of a label
3. Pushing predictions to Kili
4. Visualizing predictions in Kili

## 1. Understanding the different types of labels
A label is the annotation or combination of all annotations created on an asset. For example, all houses identified on the satellite image, or all the information annotated text on the document.

There are three categories of labels:

- **default**: an ordinary label, made by an annotator
- **prediction**: a pre-annotation, made by a model
- **review**: a check, carried out by a reviewer

When you export data (see [How to export labels](https://github.com/kili-technology/kili-playground/blob/master/recipes/export_labels.ipynb)), you can find out which category a label belongs to by looking at the field `labelType`. It can take the following values: `PREDICTION`, `DEFAULT`, `REVIEW`.

## 2. Understanding the data model of a label
Predictions under push in Kili in the form of python dictionaries. The format of the dictionary to be pushed depends on the type of data (text, image, audio), the machine learning task(s) the project is about (e.g. simple, multiple classification, transcription, named entity recognition, object detection, etc.) and their order. In summary, it depends on the JSON format that describes the interface of your annotation project.

The following cell shows you how to view this JSON. You need to update the credentials `email`, `password` and `project_id` before.

In [ ]:
!pip install kili
from kili.authentication import KiliAuth
from kili.playground import Playground

email = 'YOUR EMAIL'
password = 'YOUR PASSWORD'
project_id = 'YOUR PROJECT ID'

kauth = KiliAuth(email=email, password=password)
playground = Playground(kauth)

tools = playground.get_tools(project_id=project_id)
assert len(tools) == 1
tool = tools[0]
print(tool['jsonSettings'])

In the same way, we will visualize the data model of a label. To do this, make sure you have labeled at least one asset in Kili.

The following cell allows you to retrieve the first label in encounters.

In [ ]:
assets = playground.get_assets(project_id=project_id)

for asset in assets:
    labels = asset['labels']
    if not labels or len(labels) == 0:
        continue
    label = labels[0]
    print(label)
    break

Taking the example of text classification task, here are the fields you will see:

- `createdAt`: Date of creation of the Label.
- `id`: Unique identifier of the Label in the Kili database.
- `jsonResponse`: List of label annotations. See detail lower.
 - `JOB_ID_1`: First annotation task, as defined in the interface builder.
  - `categories`: Category the Asset (text) belongs to.
   - `name`: Name of the category.
   - `confidence`: Category Confidence Index. The value is between 0 and 100 for a PREDICTION type Label (produced by a model). The value is always 100 for a Label created by a human.
- `labelType`: Type of label used to identify if the label is a prediction (made by a model), an ordinary label (made by an annotator) or a journal (carried out by a reviewer). Can take the following values PREDICTION, DEFAULT, REVIEW.
- `secondsToLabel`: Time spent creating this Label.
- `totalSecondsToLabel`: Time spent creating Labels for this Asset, all Labels combined.

For other types of tasks the fields will be the same except for the `jsonResponse` for which we follow more or less the data model of the Google APIs.

## 3. Pushing predictions to Kili
To make a prediction in Kili, you need 4 pieces of information:

- A project ID
- An external asset ID
- A model name (arbitrary)
- A jsonResponse, in the same format as the labels `jsonResponse` (see above)

This cell imports a prediction to the asset whose label we just saw.

In [ ]:
externalId = asset['externalId']
model_name = 'v0.0.1'
json_response = label['jsonResponse']

playground.create_predictions(
    project_id=project_id,
    external_id_array=[externalId],
    model_name_array=[model_name],
    json_response_array=[json_response])


## 4. Visualizing predictions in Kili
To verify that the prediction in question was indeed pushed in Kili, you can go to https://cloud.kili-technology.com/label/projects/[PROJECT_ID]/dataset/labels?currentPage=1&pageSize=50. You should get a new `PREDICTION` line like this:

![update settings](img/import_predictions - visualize prediction.png)

## Summary
In this tutorial, we accomplished the following:

We introduced the concept of Kili interface settings and the 3 different kind of labels (`DEFAULT`, `PREDICTION`, `REVIEW`). We demonstrated how to retrieve a `DEFAULT` label and how to create a `PREDICTION` label. If you enjoyed this tutorial, check out the other Recipes for other tutorials that you may find interesting, including demonstrations of how to use Kili.

You can also visit the Kili website or Kili documentation for more info!

